<a href="https://colab.research.google.com/github/wanjaperpetual/assignment/blob/main/machine_learing_one_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load and prepare the MNIST dataset.
The pixel values of the images range from 0
through 255.
**Scale these values to a range of 0 to 1 by dividing the values by 255.0.**
his also converts the sample data from integers to floating-point numbers:

> the sample data from integers to floating-point numbers:



In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.17.0


In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


mnist.load_data() loads the MNIST dataset. It returns two tuples:

(x_train, y_train) for the training set (images and corresponding labels).
(x_test, y_test) for the test set (images and corresponding labels).


x_train and x_test contain the images (28x28 pixel grayscale images of digits).

y_train and y_test contain the labels (which digit the image represents, i.e., a number between 0 and 9).

Build a machine learning model

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Sequential Model This creates a sequential model, which means the layers are stacked one after the other

The Flatten layer takes the 28x28 pixel image and flattens it into a 1D array (28x28 = 784 values).

This is necessary because the next layers (Dense layers) expect a flat input rather than a 2D image matrix.
input_shape=(28, 28) specifies the shape of the input image

The Flatten layer takes the 28x28 pixel image and flattens it into a 1D array (28x28 = 784 values).

This is necessary because the next layers (Dense layers) expect a flat input rather than a 2D image matrix.

input_shape=(28, 28) specifies the shape of the input image


This layer randomly "drops out" (sets to 0) 20% of the neurons in the previous layer during training.


. Dense Layer (Output Layer with 10 units
This is the output layer with 10 neurons, corresponding to the 10 possible classes (digits 0-9).
Each neuron will output a score representing the model's confidence in each digit class.
There's no activation function here because the next step is usually to apply softmax, which converts these scores into probabilities during training.





In [6]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.20474295, -0.07953407,  0.3742419 , -0.88700193,  0.13990517,
         0.371109  ,  0.908881  ,  0.3552382 , -0.0735388 , -0.0220452 ]],
      dtype=float32)

**The tf.nn.softmax function converts these logits to probabilities for each class:**

In [7]:
tf.nn.softmax(predictions).numpy()

array([[0.09871855, 0.07429145, 0.11695295, 0.03313294, 0.09252094,
        0.11658712, 0.19961905, 0.1147514 , 0.07473817, 0.07868753]],
      dtype=float32)

tf.keras.losses.SparseCategoricalCrossentropy:

This is a built-in loss function in TensorFlow used for multi-class classification problems where the target labels are integers (class indices).
Sparse categorical crossentropy is used instead of categorical crossentropy when the labels are provided as integers (e.g., 0, 1, 2, ..., n-1 for n classes), making it memory efficient because it avoids one-hot encoding of the labels.
from_logits=True:

This parameter indicates that the output of the model consists of logits (raw scores) rather than probabilities.
When from_logits=True is set, the function will automatically apply the softmax activation function internally to convert the logits to probabilities before calculating the loss. This is important because softmax helps in interpreting the raw scores as probabilities

In [8]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

y_train[:1]: This is the actual label (true target) for the first sample in the training set. Since you're using sparse categorical crossentropy, this label is represented as an integer, indicating the class (digit 0-9) that the first sample belongs to.

predictions: This contains the raw logits (unprocessed scores) output by the model for the first sample. Each of the 10 values in predictions corresponds to one of the digit classes (0-9), and these logits are not yet normalized into probabilities.

loss_fn(y_train[:1], predictions): This computes the sparse categorical cross-entropy loss between the true label (y_train[:1]) and the raw logits (predictions).

Since from_logits=True, the loss function will apply softmax internally to the logits, converting them to probabilities before calculating the loss.
The result is a scalar value representing how "wrong" the model’s prediction was for the first sample. The lower the loss, the better the prediction matches the true labe

In [9]:
loss_fn(y_train[:1], predictions).numpy()

2.1491165

Before you start training, configure and compile the model using Keras Model.compile. Set the optimizer class to adam, set the loss to the loss_fn function you defined earlier, and specify a metric to be evaluated for the model by setting the metrics parameter to accuracy

optimizer='adam':

The Adam optimizer is a popular optimization algorithm that adapts the learning rate for each parameter in the model.
loss=loss_fn:

This sets the loss function used for training. In this case, loss_fn was
metrics=['accuracy']:

This specifies that the model will track accuracy during training and evaluation.

After compiling the model, when you call model.fit() to train the model, the following steps happen:

Optimization: The Adam optimizer updates the model’s parameters to minimize the loss.
Loss Calculation: The sparse categorical cross-entropy loss is computed to evaluate how well the model’s predictions match the true labels.

Metrics Calculation: In each epoch, accuracy is calculated to provide insight into how well the model is performing at classifying the digits correctly.

In [10]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

**Train and evaluate your model**

Use the Model.fit method to adjust your model parameters and minimize the los




In [11]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8656 - loss: 0.4717
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.9551 - loss: 0.1520
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9672 - loss: 0.1096
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9725 - loss: 0.0910
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.9763 - loss: 0.0751


Training and Learning:

During each epoch, the model adjusts its weights based on the loss and the gradients computed from the training data.

The goal is to minimize the loss and improve the model’s performance on the training set.

Training typically involves multiple epochs because a single pass over the dataset is usually insufficient for the model to learn effectively.

Batch Processing:

During training, the dataset is often split into batches (groups of samples) that are passed through the network in one go.

Each time a batch is passed through the model, the weights are updated. One epoch is complete when all batches have been processed.

Overfitting and Underfitting:

Too few epochs: The model may not have learned enough from the data, leading to underfitting.

Too many epochs: The model might start to memorize the training data instead of generalizing to unseen data, leading to overfitting. M

onitoring the validation loss/accuracy can help determine when to stop training

An epoch refers to one complete pass of the entire training dataset through the neural network during the training process

s used to evaluate the trained model on a test dataset. It returns the loss value and any other metrics (such as accuracy) that you have specified when compiling the model.

Explanation:

x_test: This is the test dataset's input features (in the case of MNIST, it would be images of digits).

y_test: These are the true labels (target outputs) for the test dataset (the digit classes corresponding to the images in x_test).

verbose=2: Controls the verbosity level of the output:

verbose=0: No output will be printed.

verbose=1: A progress bar is displayed for each batch during evaluation.

verbose=2: A more compact, one-line summary of the evaluation results is printed.

In [12]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - 2ms/step - accuracy: 0.9736 - loss: 0.0790


[0.07898963242769241, 0.9735999703407288]

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it

Softmax is an activation function commonly used in the output layer of a neural network for multi-class classification problems. It converts raw logits (unprocessed scores) into probabilities, where each probability represents the likelihood of the input belonging to a particular class.

Key Properties:
The softmax function squashes the output of each class into a range between 0 and 1.
The sum of all output probabilities is 1, making it ideal for classification tasks where the goal is to assign a class label to the input.

In [13]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [14]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[7.6012151e-07, 1.2137344e-09, 5.8612272e-06, 5.4495427e-04,
        1.6575603e-10, 1.5939419e-06, 5.6553801e-14, 9.9938595e-01,
        3.9325298e-07, 6.0498580e-05],
       [6.8780852e-09, 8.9505775e-06, 9.9997532e-01, 1.5618796e-05,
        3.5931680e-14, 8.8104080e-08, 1.7298192e-09, 8.6058860e-12,
        2.4655716e-08, 5.5449731e-13],
       [4.5905045e-07, 9.9836773e-01, 7.0653768e-04, 7.5473305e-05,
        9.7490018e-05, 5.2874620e-06, 4.4635541e-05, 4.6686374e-04,
        2.3328899e-04, 2.1829344e-06],
       [9.9976844e-01, 2.3651201e-09, 6.3225001e-05, 5.8514761e-08,
        1.4860895e-07, 1.4877529e-05, 1.3944253e-04, 9.3795343e-06,
        8.8649594e-09, 4.4848362e-06],
       [5.4169399e-05, 3.5773917e-09, 2.3314383e-04, 3.5825269e-07,
        9.9387276e-01, 2.1449199e-05, 2.3130224e-05, 1.6415313e-04,
        2.8098921e-06, 5.6280009e-03]], dtype=float32)>